In [39]:
import torch
import numpy as np
import pandas as pd
from src.preprocessing import remove_stop_words
from src.model_architectures.characterbert_classifier import SiameseNetwork

In [40]:
net = SiameseNetwork()

In [112]:
df = pd.read_csv('data/train/spec_train_data_new.csv')

In [192]:
label_0 = [0, 0, 0]
label_1 = [0, 0, 0]
token = '256 gb'
for row in df.iloc:
    if token in row.title_one and token in row.title_two:
        if row.label == 1:
            label_1[0] += 1
        else:
            label_0[0] += 1
    
    elif token in row.title_one and token not in row.title_two:
        if row.label == 1:
            label_1[1] += 1
        else:
            label_0[1] += 1
    
    elif token not in row.title_one and token in row.title_two:
        if row.label == 1:
            label_1[2] += 1
        else:
            label_0[2] += 1

print("In both titles: ", label_0[0], label_1[0])
print("In title one, not title 2: ", label_0[1], label_1[1])
print("In title two not title 1: ", label_0[2], label_1[2])

In both titles:  370 481
In title one, not title 2:  111 0
In title two not title 1:  125 0


In [41]:
net.load_state_dict(torch.load('./models/Test/test_epoch2.pt'))
#net = torch.load('./models/BERT/0.2.1.1_BERT_epoch_3.pt')
net.eval()
print('Loaded Model')

Loaded Model


In [12]:
def inference():
    title1 = input('First title: ')
    title2 = input('Second title: ')
    
    title1 = remove_stop_words(title1)
    title2 = remove_stop_words(title2)
    
    data = np.array([title1, title2]).reshape(1, 2)
    forward = net(data)
    np_forward = forward.detach().numpy()[0]
    
    print('Output: {}'.format(torch.argmax(forward)))
    print('Softmax: Negative {:.4f}%, Positive {:.4f}%'.format(np_forward[0], np_forward[1]))

In [57]:
inference()

First title: ASUS - ZenBook 14"" Ultra-Slim FHD Notebook - i7-1065G7 - 8GB 512GB in Pine Grey - Pine Grey
Second title: ASUS - ZenBook 14"" Ultra-Slim FHD Notebook - i7-1065G7 - 8GB 512GB in Pine Grey - Pine Grey windows thunderbolt nanoedge
Output: 1
Softmax: Negative 0.0000%, Positive 1.0000%


In [54]:
inference()

First title: ASUS ZenBook 14 Ultra-Slim Laptop 14" Full HD NanoEdge Bezel, Intel Core i7-1065G7, 8 GB RAM, 512 GB PCIe SSD, NumberPad, Thunderbolt, Windows Home, Pine Grey
Second title: ASUS - ZenBook 14"" Ultra-Slim FHD Notebook - i7-1065G7 - 8 GB 512 GB in Pine Grey - Pine Grey
Output: 0
Softmax: Negative 1.0000%, Positive 0.0000%


In [66]:
inference()

First title: Lenovo Yoga C740 2-in-1 14" Full HD 1080p Touchscreen Laptop PC, Intel Core i5-10210U Quad Core Processor, 8GB DDR4 RAM, 256GB SSD, Backlit Keyboard, Webcam, WiFi, Bluetooth, Windows 10, Mica
Second title: "Lenovo Yoga 14.0"" Full HD 2-in-1 Touchscreen Notebook, 10th Gen Intel Core i5-10210U, 8GB DDR4, 256GB SSD, Intel UHD Graphics, Wifi-AC, Bluetooth, USB, Fingerprint Reader, Windows 10
Output: 0
Softmax: Negative 1.0000%, Positive 0.0000%


In [68]:
inference()

First title: 20 + 10
Second title: 30
Output: 0
Softmax: Negative 1.0000%, Positive 0.0000%


In [268]:
inference()

First title: 1 tb
Second title: 2 tb
Output: 1
Softmax: Negative 0.0015%, Positive 0.9985%


In [ ]:
hp 15 4 inch laptop intel core i7 4770k 3 5 ghz 480 gb hard drive 104 gb memory

hp chromebook 15 4"" intel core i7 4770k cpu 480 gb hdd 104 gb ram

In [ ]:
16 gb ram 512 gb ssd asus zenbook ryzen cpu